In [29]:
import pandas as pd
import numpy as np
import gradio as gr
import nltk
import spacy
import pickle
from gensim.models.ldamodel import LdaModel
from gensim import corpora
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nlp = spacy.load("en_core_web_sm")
nltk.download('vader_lexicon')

from absa_functions import *

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\echeng\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
def parse_targets(nlp, review):
    doc = nlp(review)
    targets = []
    target = ''

    for token in doc:
        if (token.dep_ in ['nsubj','dobj']) and (token.pos_ =='NOUN' or token.pos_ == 'PROPN'):
            target = token.text
            targets.append(target)

    return targets

In [3]:
def parse_adjectives(nlp, review):
    doc = nlp(review)
    adjectives = []
    adjective = ''

    for token in doc:
        if token.pos_ == 'ADJ' and token.dep_ != 'amod':
            prepend = ''
            for child in token.children:
                if child.pos_ != 'ADV':
                    continue
                prepend += child.text + ' '
            adjective = prepend + token.text
            adjectives.append(adjective)

    return adjectives

In [9]:
def get_topic_from_word(word, lda_model, topic_map):
    topics_raw = lda_model.get_term_topics(word, minimum_probability=0.0000001)
    topic_dict = {topic_map[tup[0]]: tup[1] for tup in topics_raw}
    best_topic = max(topic_dict, key=topic_dict.get)

    return best_topic

In [7]:
corpus = pickle.load(open('corpus.pkl', 'rb'))
dictionary = corpora.Dictionary.load('dictionary.gensim')
lda_model = LdaModel.load('best_lda_model.gensim')

In [15]:
sid = SentimentIntensityAnalyzer()

In [8]:
#numerical mapping for topics in LDA model
topic_map = {0: 'menu', 1: 'service', 2: 'miscellaneous', 3: 'place', 4: 'price', 5: 'food', 6: 'staff'}

In [42]:
review = 'Fabulous dinner & environment but the older waiters have a real sense of entitlement.'

In [43]:
parse_targets(nlp, review)

['dinner', 'waiters', 'sense']

In [44]:
parse_adjectives(nlp, review)

[]

In [47]:
def pos_prediction(restaurant_review):
    nlp = spacy.load("en_core_web_sm")
    targets = parse_targets(nlp, restaurant_review)
    adjectives = parse_adjectives(nlp, restaurant_review)

    outputs = []
    if len(targets) == len(adjectives): 
        for i in range(0, len(targets)):
            output = {}
            
            output.update({'aspect': targets[i], 'adjective': adjectives[i]})

            topic = get_topic_from_word(prepare_text_for_lda(targets[i])[0], lda_model, topic_map)
            score = sid.polarity_scores(adjectives[i])['compound']
            sentiment = 'positive' if score > 0 else ('neutral' if score == 0 else 'negative')
            output.update({'topic': topic, 'polarity': sentiment})
            outputs.append(output)
    elif len(targets) > len(adjectives):
        for i in range(0, len(targets)):
            output = {}
            try:
                output.update({'aspect': targets[i], 'adjective': adjectives[i]})
            except IndexError:
                output.update({'aspect': targets[i], 'adjective': 'None'})
            outputs.append(output)
    elif len(targets) < len(adjectives):
        for i in range(0, len(adjectives)):
            output = {}
            try:
                output.update({'aspect': targets[i], 'adjective': adjectives[i]})
            except IndexError:
                output.update({'aspect': 'None', 'adjective': adjectives[i]})
            outputs.append(output)
    
    return pd.DataFrame(outputs)
        


In [48]:
pos_prediction(review)

,aspect,adjective
0,dinner,None
1,waiters,None
2,sense,None


In [41]:
iface = gr.Interface(fn=pos_prediction, 
                    inputs=gr.inputs.Textbox(lines=2, placeholder='Enter restaurant review here...'),
                    outputs=gr.outputs.Dataframe(headers=['Aspect','Adjective', 'Topic', 'Polarity']),
                    examples=[
                        ['The restaurant is too dark, and the bathroom was not clean. Also, everyone there is rude.'],
                        ['Fabulous dinner & environment but the older waiters have a real sense of entitlement.'],
                        ['Entrees were way too expensive.'],
                    ])
iface.launch(share=True)

Running locally at: http://127.0.0.1:7865/
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://13084.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7865/',
 'https://13084.gradio.app')

[2021-09-30 19:32:24,110] ERROR in app: Exception on /api/predict/ [POST]
Traceback (most recent call last):
  File "C:\Users\echeng\Anaconda3\envs\absa_env\lib\site-packages\flask\app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\echeng\Anaconda3\envs\absa_env\lib\site-packages\flask\app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\echeng\Anaconda3\envs\absa_env\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "C:\Users\echeng\Anaconda3\envs\absa_env\lib\site-packages\flask\app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\echeng\Anaconda3\envs\absa_env\lib\site-packages\flask\app.py", line 1499, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\echeng\Anaconda3\envs\absa_env\lib\s